In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/event_extraction/

/content/drive/MyDrive/event_extraction


In [3]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm
from torch.utils.data import DataLoader
import torch
import os
from src.eventtriplet_dataset import EventTripletDataset
from src.eventtype_finetune import EventRetrieverFineTune, EventRetrieverTrainer
from src.utils.device_util import getDeviceInfo
from src.utils.data_utils import build_label_maps, build_labels
from src.eventtype_retriever import EventTypeRetriever
from src.wikievents_dataset import WikiEventsSentenceDataset

In [4]:
device = getDeviceInfo()
print(f"Device info::: {device}")

MODEL_NAME = "roberta-base"

MAX_LENGTH = 256
BATCH_SIZE = 16
LEARNING_RATE = 1e-5
EPOCHS = 20

# CONTEXT_PATH = ""
CONTEXT_PATH = '/content/drive/MyDrive/event_extraction/'
CHECKPOINT_DIR = f"{CONTEXT_PATH}checkpoints"

TRAIN_JSON_PATH = f"{CONTEXT_PATH}data/train.jsonl"
VAL_JSON_PATH = f"{CONTEXT_PATH}data/dev.jsonl"
TEST_JSON_PATH = f"{CONTEXT_PATH}data/test.jsonl"

LABEL_CACHE_PATH = f"{CONTEXT_PATH}processing_data/event_types.json"
TRAIN_CACHE_PATH = f"{CONTEXT_PATH}processing_data/train.json"
VAL_CACHE_PATH = f"{CONTEXT_PATH}processing_data/dev.json"
TEST_CACHE_PATH = f"{CONTEXT_PATH}processing_data/test.json"

#get event type
event_types = build_labels(TRAIN_JSON_PATH, LABEL_CACHE_PATH)


Device info::: cuda
Loaded labels from /content/drive/MyDrive/event_extraction/processing_data/event_types.json


In [5]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
special_tokens = ["<tgr>"]
tokenizer.add_tokens(special_tokens)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

1

In [6]:

# Dataset & Dataloader
#file_path, tokenizer, label2id, max_length=128, cache_path=None
train_dataset = WikiEventsSentenceDataset(TRAIN_JSON_PATH, tokenizer, MAX_LENGTH, TRAIN_CACHE_PATH)
val_dataset = WikiEventsSentenceDataset(VAL_JSON_PATH, tokenizer, MAX_LENGTH, VAL_CACHE_PATH)
test_dataset = WikiEventsSentenceDataset(TEST_JSON_PATH, tokenizer, MAX_LENGTH, TEST_CACHE_PATH)

train_triplet_dataset = EventTripletDataset(train_dataset, event_types, tokenizer, MAX_LENGTH)
val_triplet_dataset = EventTripletDataset(val_dataset, event_types, tokenizer, MAX_LENGTH)
test_triplet_dataset = EventTripletDataset(test_dataset, event_types, tokenizer, MAX_LENGTH)

train_loader = DataLoader(train_triplet_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_triplet_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_triplet_dataset, batch_size=BATCH_SIZE, shuffle=False)


Loading processed dataset from /content/drive/MyDrive/event_extraction/processing_data/train.json
Loading processed dataset from /content/drive/MyDrive/event_extraction/processing_data/dev.json
Loading processed dataset from /content/drive/MyDrive/event_extraction/processing_data/test.json


In [7]:
sentence = "Roadside IED <tgr> kills </tgr> Russian major general in Syria"
top_k = 3

retriever = EventTypeRetriever(model_name=MODEL_NAME, device=device, event_types=event_types, tokenizer=tokenizer, max_length=MAX_LENGTH)
print(retriever.retrieve(sentence, topk=top_k))

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


[('Contact.ThreatenCoerce.Correspondence', 0.9776907563209534), ('Contact.ThreatenCoerce.Unspecified', 0.9742143750190735), ('Cognitive.Research.Unspecified', 0.9669650793075562)]


In [8]:
model = EventRetrieverFineTune(MODEL_NAME)
model.encoder.resize_token_embeddings(len(tokenizer))

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(50266, 768, padding_idx=1)

In [9]:
trainer = EventRetrieverTrainer(
    model = model,
    tokenizer = tokenizer,
    train_loader=train_loader,
    val_loader = val_loader,
    event_types = event_types,
    device = device,
    batch_size = BATCH_SIZE,
    lr = LEARNING_RATE,
    epochs = EPOCHS,
    checkpoint_dir = CHECKPOINT_DIR
)

trainer.train()

Epoch 1, Train Loss: 0.4983, Val Loss: 0.4605
Saved best model to /content/drive/MyDrive/event_extraction/checkpoints/retrieve_best_model (val loss 0.4605)


Epoch 2, Train Loss: 0.3708, Val Loss: 0.3786
Saved best model to /content/drive/MyDrive/event_extraction/checkpoints/retrieve_best_model (val loss 0.3786)


Epoch 3, Train Loss: 0.2349, Val Loss: 0.2248
Saved best model to /content/drive/MyDrive/event_extraction/checkpoints/retrieve_best_model (val loss 0.2248)


Epoch 4, Train Loss: 0.1586, Val Loss: 0.1749
Saved best model to /content/drive/MyDrive/event_extraction/checkpoints/retrieve_best_model (val loss 0.1749)


Epoch 5, Train Loss: 0.1252, Val Loss: 0.1972


...

Epoch 16, Train Loss: 0.0519, Val Loss: 0.1043
Saved best model to /content/drive/MyDrive/event_extraction/checkpoints/retrieve_best_model (val loss 0.1043)


Epoch 17, Train Loss: 0.0550, Val Loss: 0.1465


Epoch 18, Train Loss: 0.0476, Val Loss: 0.1347


Epoch 19, Train Loss: 0.0488, Val Loss: 0.0879
Saved best model to /content/drive/MyDrive/event_extraction/checkpoints/retrieve_best_model (val loss 0.0879)


Epoch 20, Train Loss: 0.0459, Val Loss: 0.1225


In [10]:
avg_test_loss = trainer.test(test_loader)

retriever = EventTypeRetriever(
    model_name=f"{CHECKPOINT_DIR}/retrieve_best_model",
    device=device,
    tokenizer=tokenizer,
    event_types=event_types,
    max_length=MAX_LENGTH
)

print(retriever.retrieve(sentence, topk=top_k))

Test Loss: 0.0794
[('Life.Die.Unspecified', 0.996371865272522), ('Life.Injure.Unspecified', 0.8573180437088013), ('Conflict.Attack.Unspecified', -0.013318629935383797)]
